In [8]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Access the variables using os.environ.get
openai_api_key = os.environ.get("gg")

# Print or use the variable as needed
print(f"OPENAI_API_KEY: {openai_api_key}")

OPENAI_API_KEY: haha


In [29]:
from openai import OpenAI
from getpass import getpass
import requests

# from langchain.document_loaders import PyMuPDFLoader
# from langchain.retrievers import ArxivRetriever

In [50]:
load_dotenv()

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
#     api_key= oai_key,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What do you know about OpenAI and elon musk do they have some relation?",
        }
    ],
    model="gpt-3.5-turbo",
)

In [52]:
chat_completion.choices[0].message.content

"OpenAI is an artificial intelligence research lab founded in December 2015. It was initially co-founded by Elon Musk, along with Sam Altman, Greg Brockman, Ilya Sutskever, John Schulman, and Wojciech Zaremba. OpenAI's primary goal is to develop safe and beneficial AI technologies while ensuring the broad distribution of their benefits.\n\nHowever, Elon Musk is no longer directly involved with OpenAI as of February 2018. Musk has stated that his departure was due to potential conflicts that might arise due to his involvement in Tesla, Neuralink, and SpaceX, where AI is also a significant focus. Despite stepping down from the board, Musk has expressed support for OpenAI and its mission.\n\nOpenAI has garnered attention for its innovative research and contributions in the field of artificial intelligence, including developing advanced language models such as GPT-3 (Generative Pre-trained Transformer 3)."

In [ ]:
chat_completion

# Load Data

In [19]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://arxiv.org/pdf/2311.12755.pdf")
pages = loader.load_and_split()

KeyboardInterrupt: 

In [17]:
len(pages)

31

In [8]:
pages

[Document(page_content='A ROADMAP TO YOUR JOURNEY TO FINANCIAL SECURITY  |  1Saving and Investing\nA Roadmap To Your Financial Security \nThrough Saving and Investing', metadata={'source': 'Resources/sec-guide-to-savings-and-investing.pdf', 'page': 0}),
 Document(page_content='2  |  SAVING AND INVESTING', metadata={'source': 'Resources/sec-guide-to-savings-and-investing.pdf', 'page': 1}),
 Document(page_content='A ROADMAP TO YOUR JOURNEY TO FINANCIAL SECURITY  |  1\nDear Reader\nWhile money doesn’t grow on trees, it can grow when \nyou save and invest wisely.\nKnowing how to secure your financial well-being is one \nof the most important things you’ll ever need in life. Y ou don’t have to be a genius to do it. Y ou just need to know a few basics, form a plan, and be ready to stick to it. No matter how much or little money you have, the important thing is to educate yourself about your opportunities. In this brochure, we’ll cover the basics on saving and investing.\nAt the SEC, we enfor

## HTML

In [10]:
from langchain.document_loaders import UnstructuredURLLoader

urls = [
    "https://hbr.org/2021/08/how-to-make-smart-investments-a-beginners-guide",
    "https://www.nerdwallet.com/article/investing/how-to-invest-money",
    "https://www.nerdwallet.com/article/investing/investment-strategies"

]

loader = UnstructuredURLLoader(urls=urls)

data = loader.load()

In [14]:
data[1]

Document(page_content="MORE LIKE THIS\n\nInvestingInvesting Planning & Strategy\n\nInvesting money in the stock market is one of the main ways to build wealth and save for long-term goals such as retirement. But figuring out the best strategy to invest that money can feel daunting. That doesn't need to be the case, though —\xa0there are several straightforward, beginner-friendly ways to invest.\n\nAdvertisement\n\nCharles Schwab Interactive Brokers IBKR Lite Webull NerdWallet rating\xa0 NerdWallet's ratings are determined by our editorial team. The scoring formula for online brokers and robo-advisors takes into account over 15 factors, including account fees and minimums, investment choices, customer support and mobile app capabilities. 4.9 /5 NerdWallet rating\xa0 NerdWallet's ratings are determined by our editorial team. The scoring formula for online brokers and robo-advisors takes into account over 15 factors, including account fees and minimums, investment choices, customer suppor

In [13]:
len(data)

3

In [107]:



class Scraper:
    """
    Scraper class to extract the content from the links
    """
    def __init__(self, urls):
        self.user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0"
        """
        Initialize the Scraper class.
        Args:
            urls:
        """
        self.urls = urls
        self.session = requests.Session()
        self.session.headers.update({
            "User-Agent": self.user_agent
        })

    def run(self):
        """
        Extracts the content from the links
        """
        partial_extract = partial(self.extract_data_from_link, session=self.session)
        with ThreadPoolExecutor(max_workers=20) as executor:
            contents = executor.map(partial_extract, self.urls)
        res = [content for content in contents if content['raw_content'] is not None]
        return res

    def extract_data_from_link(self, link, session):
        """
        Extracts the data from the link
        """
        content = ""
        try:
            if link.endswith(".pdf"):
                print('hhhhh', link)
                content = self.scrape_pdf_with_pymupdf(link)
            elif "arxiv.org" in link:
                doc_num = link.split("/")[-1]
                content = self.scrape_pdf_with_arxiv(doc_num)
            elif link:
                content = self.scrape_text_with_bs(link, session)

            if len(content) < 100:
                return {'url': link, 'raw_content': None}
            return {'url': link, 'raw_content': content}
        
        except Exception as e:
            return {'url': link, 'raw_content': None}

    def scrape_text_with_bs(self, link, session):
        response = session.get(link, timeout=4)
        soup = BeautifulSoup(response.content, 'lxml', from_encoding=response.encoding)

        for script_or_style in soup(["script", "style"]):
            script_or_style.extract()

        raw_content = self.get_content_from_url(soup)
        lines = (line.strip() for line in raw_content.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        content = "\n".join(chunk for chunk in chunks if chunk)
        return content

    def scrape_pdf_with_pymupdf(self, url) -> str:
        """Scrape a pdf with pymupdf

        Args:
            url (str): The url of the pdf to scrape

        Returns:
            str: The text scraped from the pdf
        """
        
        print('haha ', url)
        loader = PyMuPDFLoader(url)
        doc = loader.load()
        
        print('DOCCCC', doc)
        return str(doc)

    def scrape_pdf_with_arxiv(self, query) -> str:
        """Scrape a pdf with arxiv
        default document length of 70000 about ~15 pages or None for no limit

        Args:
            query (str): The query to search for

        Returns:
            str: The text scraped from the pdf
        """
        
        print('yo')
        retriever = ArxivRetriever(load_max_docs=2, doc_content_chars_max=None)
        docs = retriever.get_relevant_documents(query=query)
        return docs[0].page_content

    def get_content_from_url(self, soup):
        """Get the text from the soup

        Args:
            soup (BeautifulSoup): The soup to get the text from

        Returns:
            str: The text from the soup
        """
        text = ""
        tags = ['p', 'h1', 'h2', 'h3', 'h4', 'h5']
        for element in soup.find_all(tags):  # Find all the <p> elements
            text += element.text + "\n"
        return text

In [109]:
scraping = Scraper(['https://arxiv.org/pdf/2311.12755.pdf'])
data = scraping.run()
print(data)


hhhhh https://arxiv.org/pdf/2311.12755.pdf
haha  https://arxiv.org/pdf/2311.12755.pdf
[]


In [100]:
len(data)

2

In [97]:
data[0]

{'url': 'https://hbr.org/2021/08/how-to-make-smart-investments-a-beginners-guide',
 'raw_content': 'Your Cart\nMaking Smart Investments: A Beginner’s Guide\nIf you make smart decisions and invest in the right places, you can reduce the risk factor, increase the reward factor, and generate meaningful returns.\xa0Here are a few questions to consider as you get started.\nWhere your work meets your life. See more from Ascend here.\nAre you a saver or spender?\nIf you went with the former, then you’re in the majority. According to a 2019 Charles Schwab survey, around 59% of Americans said they considered themselves\xa0savers. Compare that to more recent findings, however, and you’ll see\xa0that\xa063% of respondents\xa0in a similar demographic are currently living paycheck to paycheck.\nClearly, there’s a disconnect between the financial goals we are setting and the steps we are taking to realize them.\nMany of us are taught from a young age that saving is the most direct path to building w

In [12]:
loader = PyMuPDFLoader("https://www.sec.gov/investor/pubs/sec-guide-to-savings-and-investing.pdf")
doc = loader.load()

print('DOCCCC', doc)


ValueError: Check the url of your file; returned status code 403

In [114]:
len(doc)

36

In [24]:
from concurrent.futures.thread import ThreadPoolExecutor
from langchain.document_loaders import PyMuPDFLoader
from langchain.retrievers import ArxivRetriever
from functools import partial
import requests
from bs4 import BeautifulSoup

class Scrape:
    """Scraper class to scrape contents
    """
    def __init__(self, urls):
        self. urls = urls
        self.session = requests.Session()
        self.session.headers.update({
            "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0"
        })
        
        
    
    def scrape_text_with_bs(self, link, session) -> str:
        """Scraping text using beautifulSoup
        """
        
        response = session.get(link, timeout=4)
        soup = BeautifulSoup(response.content, 'lxml', from_encoding=response.encoding)

        
        for script_or_style in soup(["script", "style"]):
            script_or_style.extract() #remove all the styles and scripts
            
        
        raw_content = self.get_content_from_url(soup)
        
        lines = (line.strip() for line in raw_content.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        content = "\n".join(chunk for chunk in chunks if chunk)
        return content

        
        #print(raw_content)
    
    def get_content_from_url(self, soup):
        """Scrape the text from p,and h tags
        """
        
        text = ""
        tags = ['p', 'h1', 'h2', 'h3', 'h4', 'h5']
        for element in soup.find_all(tags):  # Find all the <p> elements
            
            print(f'{element.text} \n')
            text += element.text + "\n"
        return text
        
        
        
    def extract_data_from_link(self, link, session):
        """
        Extracts the data from the link
        """
        content = ""
        try:
            if link.endswith(".pdf"):
                content = self.scrape_pdf_with_pymupdf(link)
                
                
            elif "arxiv.org" in link:
                doc_num = link.split("/")[-1]
                content = self.scrape_pdf_with_arxiv(doc_num)
            elif link:
                content = self.scrape_text_with_bs(link, session)

            if len(content) < 100:
                return {'url': link, 'raw_content': None}
            return {'url': link, 'raw_content': content}
        except Exception as e:
            return {'url': link, 'raw_content': None}

    
    
    def scrape_pdf_with_pymupdf(self, url) -> str:
        """Scrape a pdf with pymupdf

        Args:
            url (str): The url of the pdf to scrape

        Returns:
            str: The text scraped from the pdf
        """
        
        print('exec')
        try:
            
            loader = PyMuPDFLoader(url)
            doc = loader.load()
            return str(doc)
        except Exception as e:
            print('Error', e)

    def scrape_pdf_with_arxiv(self, query) -> str:
        """Scrape a pdf with arxiv
        default document length of 70000 about ~15 pages or None for no limit

        Args:
            query (str): The query to search for

        Returns:
            str: The text scraped from the pdf
        """
        retriever = ArxivRetriever(load_max_docs=2, doc_content_chars_max=None)
        docs = retriever.get_relevant_documents(query=query)
        return docs[0].page_content
    
    
        
    def run(self):
        """Extracts the content from the links
        """
        
        #partial function
        partial_extract = partial(self.extract_data_from_link, session = self.session)
        
        with ThreadPoolExecutor(max_workers=20) as executor:
            contents = executor.map(partial_extract, self.urls)
        res = [content for content in contents if content['raw_content'] is not None]
        return res

In [25]:
scraping = Scrape(['https://www.sec.gov/investor/pubs/sec-guide-to-savings-and-investing.pdf'])

In [26]:
scraping.run()

exec
Error Check the url of your file; returned status code 403


[]

In [31]:
import asyncio

async def task_with_delay(name, delay):
    print(f"Task '{name}' started")
    await asyncio.sleep(delay)
    print(f"Task '{name}' completed after {delay} seconds")

async def main():
    # Create asynchronous tasks with different delays
    task1 = task_with_delay("Task 1", 3)
    task2 = task_with_delay("Task 2", 2)
    task3 = task_with_delay("Task 3", 1)

    # Start the tasks concurrently
    await asyncio.gather(task1, task2, task3)

# Run the asynchronous program


if __name__ == '__main__':
    asyncio.run(main())


/Users/cohlem/anaconda3/envs/okprofessor/lib/python3.9/genericpath.py:77: RuntimeWarning: coroutine 'main' was never awaited
  m = tuple(map(os.fspath, m))


RuntimeError: asyncio.run() cannot be called from a running event loop

In [33]:
import asyncio  
async def main():  
    print ("Waiting 5 seconds. ")  
    for _ in range(5):  
        await asyncio.sleep(1)  
        print ("Hello")  
    print ("Finished waiting.")  
asyncio.run(main())  

RuntimeError: asyncio.run() cannot be called from a running event loop

In [37]:
from langsmith.client import Client
import os
from dotenv import load_dotenv

from langchain_benchmarks.rag import get_eval_config

In [41]:
from langchain.chat_models import ChatOpenAI
eval_llm = ChatOpenAI(model="gpt-3.5-turbo", model_kwargs={"seed": 42})

In [42]:
load_dotenv()
get_eval_config(eval_llm)

TypeError: get_eval_config() takes 0 positional arguments but 1 was given

In [102]:
async def create_chat_completion(messages, temperature=0, stream=False, model='gpt-3.5-turbo'):
    client = OpenAI(
        # defaults to os.environ.get("OPENAI_API_KEY")
        #     api_key= oai_key,
    )

    if not stream:

        chat_completion = client.chat.completions.create(
            messages=messages,
            model=model,
            temperature=temperature
        )

        return chat_completion.choices[0].message.content


    
    



## Main run

In [6]:
import sys
sys.path.append("/Users/cohlem/Projects/OkProfessor")

from researcher.config import Config

In [10]:
from researcher.config import Config

In [11]:
ss = Config()

In [12]:
ss.llm

'gpt-3.5-turbo'

In [65]:
from colorama import Fore, Style
print(f"{Fore.RED}Error in choose_agent: {e}{Style.RESET_ALL}")

Error in choose_agent: 


In [86]:
auto_agent_instructions()

'\n        This task involves researching a given topic, regardless of its complexity or the availability of a definitive answer. The research is conducted by a specific server, defined by its type and role, with each server requiring distinct instructions.\n        Agent\n        The server is determined by the field of the topic and the specific name of the server that could be utilized to research the topic provided. Agents are categorized by their area of expertise, and each server type is associated with a corresponding emoji.\n\n        examples:\n        task: "should I invest in apple stocks?"\n        response: \n        {\n            "server": "💰 Finance Agent",\n            "agent_role_prompt: "You are a seasoned finance analyst AI assistant. Your primary goal is to compose comprehensive, astute, impartial, and methodically arranged financial reports based on provided data and trends."\n        }\n        task: "could reselling sneakers become profitable?"\n        response

In [78]:
from colorama import Fore, Style
from researcher.prompts.prompts import *

async def choose_agent(query, model):
    
    try:
        
        response = await create_chat_completion(
            messages = [
                    {"role": "system", "content": f"{auto_agent_instructions()}"},
                    {"role": "user", "content": f"task: {query}"}], 
            model=model
        )
        
        agent = json.loads(response)
        
        return agent['server'], agent['agent_role_prompt']
    
    except Exception as e:

        print(f"{Fore.RED} Error in choose_agent: {e}{Style.RESET_ALL}")
        return { "server": "Default Agent",
                "agent_role_prompt": "You are an AI critical thinker research assistant. Your sole purpose is to write well written, critically acclaimed, objective and structured reports on given text."}
    

In [87]:
ans = await choose_agent('Space exploration', model='gpt-3.5-turbo')

In [17]:
ans

NameError: name 'ans' is not defined

In [83]:
import json

In [85]:
json.loads(ans)['agent_role_prompt']

'You are a seasoned finance analyst AI assistant. Your primary goal is to compose comprehensive, astute, impartial, and methodically arranged financial reports based on provided data and trends.'

In [110]:
class Config:
    def __init__(self):

        self.retriever = "Weaviate"
        self.search_engine = "duckduckgo"
        self.llm = "gpt-3.5-turbo"
        self.max_search_query = 3
        self.max_search_results_per_query = 5
        self.total_words = 1000
        self.temperature = 0
        
        


In [25]:
import sys
sys.path

['/Users/cohlem/Projects/OkProfessor/researcher/notebooks',
 '/Users/cohlem/anaconda3/envs/okprofessor/lib/python39.zip',
 '/Users/cohlem/anaconda3/envs/okprofessor/lib/python3.9',
 '/Users/cohlem/anaconda3/envs/okprofessor/lib/python3.9/lib-dynload',
 '',
 '/Users/cohlem/anaconda3/envs/okprofessor/lib/python3.9/site-packages',
 '/Users/cohlem/Projects/OkProfessor',
 '/Users/cohlem/Projects/OkProfessor']

In [106]:
messages = [
    {"role": "system" , "content" : role },
    {"role": "user" , "content" : generate_search_queries_prompt(query) }
]

try:
    
    response = await create_chat_completion(messages, temperature =0, model ='gpt-3.5-turbo')
except Exception as e:
    print(e)
print(response)

["Benefits and risks of investing in real estate properties", "Real estate market trends and forecasts November 2023", "How to evaluate the potential return on investment in real estate properties"]


In [114]:
from researcher.prompts.prompts import *
import json 

async def get_sub_queries(query, agent_role, cfg):
    
    messages = [
        {"role": "system" , "content" : f"{agent_role}" },
        {"role": "user" , "content" : f"{generate_search_queries_prompt(query)}" }
    ]
    

    
    response = await create_chat_completion(messages, temperature=cfg.temperature, model=cfg.llm)
    response = json.loads(response)
    
    return response
    

In [112]:
sample_config = Config()

In [113]:
sample_config.temperature

0

In [115]:
query = 'investment in real estate properties'
role = "You are an AI critical thinker research assistant. Your sole purpose is to write well written, critically acclaimed, objective and structured reports on given text."

ans = await get_sub_queries(query, role, sample_config)

In [117]:
ans

['Benefits and risks of investing in real estate properties',
 'Real estate market trends and forecasts November 2023',
 'How to evaluate the potential return on investment in real estate properties']

In [123]:
from researcher.config import Config
from researcher.search.duckduckgo import Duckduckgo
from researcher.utils.functions import * 
from researcher.retriever.langchain_hybrid_retriever import HybridRetriever

class Researcher:
    def __init__(self, query ):
        self.query = query
        self.cfg = Config()
        self.agent = None
        self.role = None
        
    async def run(self):
        """
        Run the researcher
        """
        if self.cfg.search_engine == 'Duckduckgo':
            retriever = Duckduckgo()
            
        print(f'Running research for query: {self.query}')
        self.agent, self.role = await choose_agent(self.query, self.cfg.llm )
        print(f'Running {self.agent} ...')
        
        #query modification
        sub_queries = await get_sub_queries(self.query, self.role, self.cfg) + [self.query]
        
        return sub_queries
        

        
        
        
        
        

In [124]:
r = Researcher('buying real estate properties and airbnb houses')

In [122]:
generate_search_queries_prompt('ahha')

'Write 3 google search queries to search online that form an objective opinion from the following: "ahha"Use the current date if needed: November 30, 2023.\nYou must respond with a list of strings in the following format: ["query 1", "query 2", "query 3"].'

In [121]:
res = await r.run()

Running research for query: buying real estate properties and airbnb houses
Running 🏡 Real Estate Agent ...


NameError: name 'generate_search_queries_prompt' is not defined

In [74]:
res

<coroutine object Researcher.run at 0x11a554740>

In [61]:
import time

async def sample(seconds):
    time.sleep(seconds)
    print(f'{seconds}')

In [63]:
async def run_fn():
    
    print('starting')
    await sample(5)
    
    print('break')
    
    await sample(1)
    
    

In [64]:
await run_fn()

starting
5
break
1


In [60]:
time.sleep(4)

In [12]:
from openai import OpenAI
from typing import AsyncGenerator, NoReturn

client = OpenAI()

In [14]:

"""
OpenAI Response
"""
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": (
                "You are a helpful assistant, skilled in explaining "
                "complex concepts in simple terms."
            ),
        },
        {
            "role": "user",
            "content": 'heyy',
        },
    ],
    stream=True,
)

all_content = ""


for chunk in response:
    output = chunk.choices[0].delta.content, chunk.choices[0].finish_reason
    print(output)

('', None)
('Hello', None)
('!', None)
(' How', None)
(' can', None)
(' I', None)
(' assist', None)
(' you', None)
(' today', None)
('?', None)
(None, 'stop')


TypeError: object async_generator can't be used in 'await' expression